In [ ]:
from joblib import load
import librosa
import numpy as np

model = load("model0_with_best_params.joblib")
pca = load("pca_transform.joblib")
duygular = [
    'angry',
    'calm',
    'disgust',
    'fear',
    'fearful',     
    'happy',
    'neutral',
    'ps',          
    'sad',
    'surprised'
]


In [ ]:
def extract_mfcc_from_file(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    y, _ = librosa.effects.trim(y)
    y = librosa.util.normalize(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    if mfcc.shape[1] >= 100:
        mfcc = mfcc[:, :100]
    else:
        pad_width = 100 - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')
    return mfcc


In [ ]:
def extract_label_from_filename(filename):
    emotion_map = {
        '01': 'neutral',
        '02': 'calm',
        '03': 'happy',
        '04': 'sad',
        '05': 'angry',
        '06': 'fearful',
        '07': 'disgust',
        '08': 'surprised'
    }

    filename = filename.lower()
    if '-' in filename and filename.endswith('.wav'):
        
            parts = filename.split('-')
            emotion_code = parts[2]
            return emotion_map.get(emotion_code)
       

    for emotion in duygular:
        if emotion in filename:
            return emotion



In [ ]:
from sklearn.preprocessing import LabelEncoder


def get_label_encoder():
    duygular = [
    'angry',
    'calm',
    'disgust',
    'fear',
    'fearful',     
    'happy',
    'neutral',
    'ps',         
    'sad',
    'surprised'
]
    encoder = LabelEncoder()
    encoder.fit(duygular)
    return encoder

In [ ]:
def predict_single_file(file_path, model, pca, label_encoder):
    true_label = extract_label_from_filename(file_path)
    mfcc = extract_mfcc_from_file(file_path)
    mfcc_reduced = pca.transform(mfcc.reshape(1, -1))  
    y_pred = model.predict(mfcc_reduced)

    print(" Tek Ses Tahmini")
    print("Gerçek Etiket:", true_label)
    print("Model Tahmini:", label_encoder.inverse_transform(y_pred)[0])


In [49]:
encoder=get_label_encoder()


predict_single_file("ses/test_10/OAF_rat_disgust.wav", model, pca, encoder)


 Tek Ses Tahmini
Gerçek Etiket: disgust
Model Tahmini: disgust
